### Load Pretrained Model
Load a pretrained Mamba Model that is compatible with Transformers Library

In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
tokenizer = AutoTokenizer.from_pretrained('Q-bert/Mamba-130M')

text = "Hi"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Hi, I'm looking for a new job. I've been working at a company for about a


In [ ]:
print(type(model))

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    
print('plain',print_trainable_parameters(model))

<class 'modeling_mamba.MambaForCausalLM'>
trainable params: 129135360 || all params: 129135360 || trainable%: 100.0
plain None


### Load LoRA adapters from the hub

In [ ]:
a1 = MambaForCausalLM.from_pretrained('mlsquare/exp-lora-ada-1')
a2 = MambaForCausalLM.from_pretrained('mlsquare/exp-lora-ada-2')

/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
a1.save_pretrained("./mbins/tmp/ada-1")
a2.save_pretrained("./mbins/tmp/ada-2")

/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


### Average the adapters based on some weight
do weighed mean of the adapters of the same type

In [ ]:
from peft import PeftMixedModel, PeftModel


base_model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
print('base mamba',print_trainable_parameters(base_model))
peft_model = PeftModel.from_pretrained(base_model, "./mbins/tmp/ada-1" , "adapter1")
print('base mamba',print_trainable_parameters(base_model))

# note this is just a copy of adapter 1 (only syntax verification)
peft_model.load_adapter("./mbins/tmp/ada-1", "adapter2")
print('two adapters loaded',print_trainable_parameters(peft_model))

# note this is just a copy of adapter 1 (only syntax verification)
peft_model.load_adapter("./mbins/tmp/ada-1", "adapter3")
print('two adapters loaded',print_trainable_parameters(peft_model))


peft_model.add_weighted_adapter(["adapter1", "adapter2","adapter3" ], [1.0,1.0,1.0], combination_type="linear", adapter_name="merged")
print('lora added',print_trainable_parameters(peft_model))

peft_model.delete_adapter("adapter1")
peft_model.delete_adapter("adapter2")
peft_model.delete_adapter("adapter3")

peft_model.merge_and_unload()
print('merged mamba',print_trainable_parameters(peft_model))


trainable params: 129135360 || all params: 129135360 || trainable%: 100.0
base mamba None
trainable params: 0 || all params: 129148288 || trainable%: 0.0
base mamba None
trainable params: 12928 || all params: 129161216 || trainable%: 0.010009196568728495
two adapters loaded None
trainable params: 12928 || all params: 129174144 || trainable%: 0.010008194828835096
two adapters loaded None
trainable params: 12928 || all params: 129187072 || trainable%: 0.01000719328943379
lora added None
trainable params: 0 || all params: 129135360 || trainable%: 0.0
merged mamba None


/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/peft/tuners/tuners_utils.py:528: UserWarning: Adapter adapter2 was active which is now deleted. Setting active adapter to adapter3.
  warnings.warn(
/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/peft/tuners/tuners_utils.py:528: UserWarning: Adapter adapter3 was active which is now deleted. Setting active adapter to merged.
  warnings.warn(


In [ ]:
text = "Hi"
input_ids = tokenizer.encode(text, return_tensors="pt")
output = peft_model.generate(input_ids=input_ids)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Hi, I'm a newbie to this, but I'm trying to get my head around the
